# IN-STK5000/9000 - Adaptive methods for data-based decision making
## Credit Project

The code to reproduce experiments can be found [here](https://github.com/gsel9/ml-society-science).

#### Syed Moeen Ali Naqvi - Geir Severin Rakh Elvatun Langberg - Markus Sverdvik Heiervang  
***

### Part 1: Banker agent
In this notebook, we display and comment on the development of our banker model, and measure it against the random banker, as well as documenting the implementations of the different methods used for the class

### Task 1 - Implementing expected utility

Our action space $\mathcal{A}$ is binary: $\mathcal{A} = \{0, 1\} = \{a_1, a_2\} = \{ \text{refuse_loan}, \text{grant_loan} \}$


To calculate the expected utility, we consider two actions: $a_1$ granting the loan or $a_2$ not granting a loan. Moreover, if granting a loan, the outcome at the end of the lending period $n$ is that it can be either fully repaid $\omega_1$ or not repaid $\omega_2$. The utility of granting a loan of $m$ credits that is also repaid is $m((1 + r)^n - 1)$, whereas, if the loan is not repaid, the utility is $-m$. In case of not granting the loan, the utility is zero. Thus, given the probability of being credit-worthy, $P(\omega_1)$, the expected utility is  


$$
    \mathbb{E}(U \mid a) = m((1 + r)^n - 1)P(\omega_1) - m(1 - P(\omega_1)).
$$

This calculation is implemented as follows


```Python
def expected_utility(self, x: pd.Series, action: int) -> float:

        if action:
            # Probability of being credit worthy.
            pi = self.predict_proba(x)

            return x["amount"] * ((1 + self.rate) ** x["duration"] - 1) * pi - x["amount"] * (1 - pi)

        return 0.0
```

### Task 2 - Implementing the fit function

We are using Random forest classifier to fit a model for calculating the probability of credit-worthiness for a creditor. Random forests (RF) construct many individual decision trees at training. Predictions from all trees are pooled to make the final prediction; the mode of the classes for classification. As they use a collection of results to make a final decision, they are referred to as Ensemble techniques.

We are using scikit-learn to implement the classifier. We have included optional hyper-parameter tuning before fitting the model.

Following is the code for fit():

```Python
    def fit(self, X: pd.DataFrame, y: pd.Series) -> None:
        if self.optimize:
            #Finding optimal paramters
            param_grid = [{
                'bootstrap' : [True],
                'max_features' : list(range(10,20,1)),
                'max_depth' : list(range(10,100,10)),
                'n_estimators' : list(range(25,150,25))
            }]

            grid_search = GridSearchCV(
                estimator = RandomForestClassifier(), param_grid = param_grid, cv = 5
            )
            grid_search.fit(X, y)
            self.classifier = RandomForestClassifier(
                random_state=self.random_state, **grid_search.best_params_
            )
        else:
            self.classifier = RandomForestClassifier(
                n_estimators=100,
                random_state=self.random_state,
                class_weight="balanced"
            )
            
        self.classifier.fit(X,y)

```

The method predict_proba() ensures that the fit() is called beforehand and predicts the probability of the loan being returned. 

Following is the code for predict_proba():

```Python
    def predict_proba(self, x: pd.Series) -> float:
        if not hasattr(self, "classifier"):
            raise ValueError("This Group4Banker instance is not fitted yet. Call 'fit' "
                             "with appropriate arguments before using this method.")

        x_reshaped = np.reshape(x.to_numpy(), (1,-1))

        return self.classifier.predict_proba(x_reshaped)[0][0]
```

We are assuming that the labelling process is correct and the labels represent the ground truth. 

### Task 3 - Get best action


Assuming that we are maximising utility, a general function would be

$$
\text{best_action}(x) = \underset{a \in \mathcal{A}}{\text{argmax}} \  \mathbb{E}(U \mid a)
$$

but since our action space is binary, it can be expressed as

$$  
\text{best_action}(x) = \begin{cases}
    1,& \text{if } \mathbb{E}(U \mid a=1) > 0\\
    0,              & \text{otherwise}
\end{cases}
$$

We can translate this into python code as such:

```Python
def get_best_action(self, x: pd.Series) -> int:
        return int(self.expected_utility(x, 1) > 0)
```

### Task 4 - Documenting the banker  

For this part, we'll be interacting with the Group4Banker in the cells below. 
Before measuring the performance, we conduct a series of unit tests to assert that each method works for a few cases

In [1]:
from group4_banker import Group4Banker
import numpy as np

# seed for reproducibility
np.random.seed(42)

In [2]:
from test_group4_banker import TestGroup4Banker
import unittest
# We'll need these arguments when running the tests in jupyter notebook
unittest.main(argv=["first-arg-is-ignored"], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 2.023s

OK


We rewrote the TestLending script into a neat command-line interface so that we can customize the programs parameters.   
This will also display progress of the training, since the classifier might take some time.  

From this, we can observe that our banker performs better than the RandomBanker

In [3]:
!python3 TestLendingV2.py ../../data/credit/D_valid.csv --n-tests 100 --seed 12 --interest-rate 0.05

r=0.05, n_tests=100, seed=12

Testing on class: RandomBanker ...
100%|█████████████████████████████████████████| 100/100 [00:06<00:00, 16.11it/s]
Results:
	Average utility: 62194332770.39753
	Average return on investment: 1662784.8621222847

Testing on class: Group4Banker ...
100%|█████████████████████████████████████████| 100/100 [02:44<00:00,  1.65s/it]
Results:
	Average utility: 154030042326.10663
	Average return on investment: 3563445.2297234936


Let's see how our banker performs on the training set

In [4]:
!python3 TestLendingV2.py ../../data/credit/D_train.csv --n-tests 100 --seed 11 --interest-rate 0.05

r=0.05, n_tests=100, seed=11

Testing on class: RandomBanker ...
100%|█████████████████████████████████████████| 100/100 [00:06<00:00, 16.21it/s]
Results:
	Average utility: 286821119298.73846
	Average return on investment: 4944685.923886744

Testing on class: Group4Banker ...
100%|█████████████████████████████████████████| 100/100 [02:41<00:00,  1.61s/it]
Results:
	Average utility: 490355107143.02966
	Average return on investment: 11223170.155701187


### Part 2: Critical evaluation of banker agent?

1. Is it possible to ensure that your policy maximises revenue? How can you take into account
the uncertainty due to the limited and/or biased data? What if you have to decide for credit
for thousands of individuals and your model is wrong? How should you take that type of
risk into account?

In general, we cannot ensure that the policy maximises revenue because the probability estimates $P(\omega_1)$ and $P(\omega_2)$ about the outcome of events $\omega_1$ and $\omega_2$ might be biased. However, from a theoretical point of view where we assume $P(\omega_1)$ and $P(\omega_2)$ are true, to maximize revenue we desire 

$$
    \mathbb{E}(U \mid a) > 0 \Rightarrow m((1 + r)^n - 1)P(\omega_1) > mP(\omega_2) \Leftrightarrow (1 + r)^n - 1 > \frac{P(\omega_2)}{P(\omega_1)}, \quad P(\omega_1) > 0
$$

To select parameters $n$ and $r$ ensuring $\mathbb{E}(U \mid a) > 0$ we can either consider 

$$
    n > \frac{\ln P(\omega_2) - \ln P(\omega_1)}{\ln (1 + r)}, \quad r, P(\omega_1), P(\omega_2) > 0
$$

for fixed $r$, or 

$$
r > \exp\left ( \frac{\ln P(\omega_2) - \ln P(\omega_1)}{n} \right ) - 1, \quad n, P(\omega_1), P(\omega_2) > 0
$$

for fixed $n$.

---


To obtain a model performance measure that is more robust than a single point estimate, one can consider a re-sampling technique - such as cross-validation of bootstraping - to derive an empirical distribution of performance. In our approach, we use bootstraping to predict the optial actions on several random subsets of the test data. This produces a matrix $\mathbf{\hat{Y}} \in \left \{ a_0, a_1 \right \}^{N \times M}$ of predicted actions on $N$ bootstrap samples of size $P$. Note that the predicted actions corresponds to the ground truths $\mathbf{Y}$ in the test data.


--- 

We use a Bayesian approach to evaluate the effect of an increasing number of samples on the model by comparing the posterior probability of rejecting $a_0$ (grant loan) when $a_1$ (don't grant loan) is true, to a ground truth derived from the test data. That is, we evaluate the model's ability to reject a loan that is unlikely to be repaid. The reference rejection rate $\hat{\mu}$ of $H_0$ is computed from the test data bootstrap samples

$$
\hat{\mu}_0 = \frac{1}{NM} \sum_{n, m} \mathbb{I}(Y_{n, m} = 0)
$$

Moreover, assuming $Y_{n, m}$ are i.i.d., we include a $1 - \delta$ confidence interval around $\hat{\mu}_0$ by using Hoeffding's inequality 
$$
    \left | \hat{\mu}_0 - \mathbb{E}\left \{ \hat{\mu}_0 \right \} \right | \leq \sqrt{\frac{\ln2 \delta}{2n}} \Rightarrow \mathbb{E}\left \{ \hat{\mu}_0 \right \} \in \left [ \hat{\mu}_0 - \sqrt{\frac{\ln2 \delta}{2n}}, \hat{\mu}_0 + \sqrt{\frac{\ln2 \delta}{2n}} \right ],
$$
bounding the expected value of $\hat{\mu}_0$. 

In experiments, we ran 200 iterations of 50 bootstrap samples for $\delta = 0.05$. The code to reproduce our experiments can be found in `action_sensitivity.py`. 

In the following figure, we compare the probability of rejecting $H_0$ given model esimtates to the reference rate $\hat{\mu}_0$ derived from test data. The figure shows that the probability of granting a loan that will not be repaid converges to $\hat{\mu}_0$ as the number of boostrap samples increases.

In [1]:
from IPython.display import IFrame

IFrame("reject_null_hypo.pdf", width=800, height=550)

To further enhance model performance, we can update our prediction model providing probability estimates by including more *Decision Trees* in the *Random Forest*. The additional trees are then trained on the test data after predictions are performed. This mechanism could be implemented as follows:

```Python
def adaptive_predict_proba(X, batch_size=0, n_extra_estimators=0):
    """Use test data to re-train the model after each decision."""
    
    y_pred = model.predict()
    
    # Add more decision trees to the model and train these using the new training data.
    model.set_params(n_estimators=int(model.n_estimators) + int(n_extra_estimators), 
                     warm_start=True)
    model.train(X)
        
    return y_pred
```

2. Does the existence of this database raise any privacy concerns? If the database was secret
(and only known by the bank), but the credit decisions were public, how would that affect
privacy? (a) Explain how you would protect the data of the people in the training set. (b)
Explain how would protect the data of the people that apply for new loans. (c) Implement
a private decision making mechanism for (b),3 and estimate the amount of loss in utility
as you change the privacy guarantee.

(Moeen)

3. Come up with more discussion worthy topics

(Markus)